<a href="https://colab.research.google.com/github/dhuvik/projects/blob/master/tulip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
local = False

In [2]:
#@title Clone github repo
import json, time, os, sys, glob

if not local:
  if not os.path.isdir("TULIP-TCR"):
    os.system("git clone -q https://github.com/barthelemymp/TULIP-TCR.git")
  sys.path.append('/content/TULIP-TCR/')

if not local:
  !pip install transformers==4.32.1 datasets  tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.32.1 which is incompatible.


In [3]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import pandas as pd
import sys

import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data._utils.collate import default_collate
#from data import TranslationDataset
from transformers import BertTokenizerFast, BertTokenizer
from transformers import BertModel, BertForMaskedLM, BertConfig, EncoderDecoderModel, BertLMHeadModel, AutoModelForSequenceClassification
from sklearn.metrics import roc_auc_score

import sys
import torch
import torch.utils.data as data
from torch.nn.utils.rnn import pad_sequence
import os


from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertOnlyMLMHead, SequenceClassifierOutput
from torch.nn import MSELoss, CrossEntropyLoss, BCEWithLogitsLoss
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import ModelOutput

from transformers import PretrainedConfig
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import add_start_docstrings, add_start_docstrings_to_model_forward, logging, replace_return_docstrings
from transformers.models.encoder_decoder.configuration_encoder_decoder import EncoderDecoderConfig
import warnings

from src.multiTrans import TulipPetal, TCRDataset, BertLastPooler, unsupervised_auc, train_unsupervised, eval_unsupervised, MyMasking, Tulip, get_logscore


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


# Load model and file

In [4]:
from google.colab import files
# uploaded = files.upload()
# filesname = list(uploaded.keys())[0]
finetunedmodel = True #@param {type:"boolean"}
with open("/content/TULIP-TCR/configs/shallow.config.json", "r") as read_file:
    print("loading hyperparameter")
    modelconfig = json.load(read_file)


loading hyperparameter


# run prediction

In [5]:
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

checkpoint = torch.load("/content/TULIP-TCR/model_weights/pytorch_model.bin")

tokenizer = AutoTokenizer.from_pretrained("/content/TULIP-TCR/aatok/")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<PAD>'})

if tokenizer.sep_token is None:
    tokenizer.add_special_tokens({'sep_token': '<MIS>'})

if tokenizer.cls_token is None:
    tokenizer.add_special_tokens({'cls_token': '<CLS>'})

if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '<EOS>'})

if tokenizer.mask_token is None:
    tokenizer.add_special_tokens({'mask_token': '<MASK>'})



from tokenizers.processors import TemplateProcessing
tokenizer._tokenizer.post_processor = TemplateProcessing(
    single="<CLS> $A <EOS>",
    pair="<CLS> $A <MIS> $B:1 <EOS>:1",
    special_tokens=[
        ("<EOS>", 2),
        ("<CLS>", 3),
        ("<MIS>", 4),
    ],
)

mhctok = AutoTokenizer.from_pretrained("/content/TULIP-TCR/mhctok/")



# mhctok = AutoTokenizer.from_pretrained("mhctok/")

vocabsize = len(tokenizer._tokenizer.get_vocab())
mhcvocabsize = len(mhctok._tokenizer.get_vocab())
print(mhcvocabsize)
print("Loading models ..")
# vocabsize = encparams["vocab_size"]

max_length = 50
encoder_config = BertConfig(vocab_size = vocabsize,
                    max_position_embeddings = max_length, # this shuold be some large value
                    num_attention_heads = modelconfig["num_attn_heads"],
                    num_hidden_layers = modelconfig["num_hidden_layers"],
                    hidden_size = modelconfig["hidden_size"],
                    type_vocab_size = 1,
                    pad_token_id =  tokenizer.pad_token_id)

encoder_config.mhc_vocab_size  =mhcvocabsize

encoderA = BertModel(config=encoder_config)
encoderB = BertModel(config=encoder_config)
encoderE = BertModel(config=encoder_config)

max_length = 100
max_length = 50
decoder_config = BertConfig(vocab_size = vocabsize,
                    max_position_embeddings = max_length, # this shuold be some large value
                    num_attention_heads = modelconfig["num_attn_heads"],
                    num_hidden_layers = modelconfig["num_hidden_layers"],
                    hidden_size = modelconfig["hidden_size"],
                    type_vocab_size = 1,
                    is_decoder=True,
                    pad_token_id =  tokenizer.pad_token_id)    # Very Important

decoder_config.add_cross_attention=True

decoderA = TulipPetal(config=decoder_config) #BertForMaskedLM
decoderA.pooler = BertLastPooler(config=decoder_config)
decoderB = TulipPetal(config=decoder_config) #BertForMaskedLM
decoderB.pooler = BertLastPooler(config=decoder_config)
decoderE = TulipPetal(config=decoder_config) #BertForMaskedLM
decoderE.pooler = BertLastPooler(config=decoder_config)


# Define encoder decoder model
model = Tulip(encoderA=encoderA,encoderB=encoderB,encoderE=encoderE, decoderA=decoderA, decoderB=decoderB, decoderE=decoderE)

model.load_state_dict(checkpoint)
model.to(device)

Using device: cuda


Using pad_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using mask_token, but it is not set yet.


55
Loading models ..
self.pad_token_id 1
self.pad_token_id 1
self.pad_token_id 1


Tulip(
  (encoderA): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 128, padding_idx=1)
      (position_embeddings): Embedding(50, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)


In [8]:
import transformers

print(transformers.__version__)

4.32.1


In [9]:
filesname = "/content/TULIP-TCR/data/VDJ_test_2.csv"
compute_auc=True
from sklearn.metrics import roc_auc_score

In [10]:
pd.read_csv(filesname)

,Unnamed: 0,CDR3b,CDR3a,peptide,MHC,binder
0,0,CASSIRASYEQYF,CAGASGNTGKLIF,GILGFVFTL,HLA-A*02:01,1
1,1,CASSIRSTDTQYF,CAGSGGGSQGNLIF,GILGFVFTL,HLA-A*02:01,1
2,2,CASSIRAADTQYF,CALDMGGGSQGNLIF,GILGFVFTL,HLA-A*02:01,1
3,3,CATSSGTGAYEQYF,CVVNAHDMRF,LLWNGPMAV,HLA-A*02:01,1
4,4,CASSDASGSIGEQFF,CAVTVSNDYKLSF,CINGVCWTV,HLA-A*02:01,1
...,...,...,...,...,...,...
6127,6127,CASSWDRVLYEQYF,CSNYKLTF,GILGFVFTL,HLA-A*02:01,0
6128,6128,CASSDRLENNEKLFF,CAVTDNNNNDMRF,GILGFVFTL,HLA-A*02:01,0
6129,6129,CASSQGQEKETQYF,CAVENYQLIW,GILGFVFTL,HLA-A*02:01,0
6130,6130,CSARDQQGQNTGELFF,CVVNRGDGLTF,GILGFVFTL,HLA-A*02:01,0


In [ ]:
target_peptidesFinal = pd.read_csv(filesname)["peptide"].unique()

for target_peptide in target_peptidesFinal:
    results = pd.DataFrame(columns=["CDR3a", "CDR3b", "peptide", "rank"])
    datasetPetideSpecific= TCRDataset(filesname, tokenizer, device,target_peptide=target_peptide, mhctok=mhctok)
    print(target_peptide)
    scores = -1*np.array(get_logscore(datasetPetideSpecific, model, ignore_index =  tokenizer.pad_token_id))
    ranks = np.argsort(np.argsort(scores))
    results["CDR3a"] = datasetPetideSpecific.alpha
    results["CDR3b"] = datasetPetideSpecific.beta
    results["peptide"] = target_peptide
    results["rank"] = ranks
    # print(results)
    if compute_auc:
      dl = torch.utils.data.DataLoader(dataset=datasetPetideSpecific, batch_size=1, shuffle=False, collate_fn=datasetPetideSpecific.all2allmhc_collate_function)
      # print(unsupervised_auc(model,dl, tokenizer.pad_token_id))
      auce = roc_auc_score(datasetPetideSpecific.binder, ranks)
      print(auce)
    else:
      print(results)
      #results.to_csv('yourpath')


Loading the data ...
GILGFVFTL
(0.691439842209073, 0.8354319526627219, 0.9531092702169626)
0.9531092702169626
Loading the data ...
LLWNGPMAV
(0.5719838955990559, 0.6018325697625988, 0.8771345272803)
0.8771345272803
Loading the data ...
CINGVCWTV
(0.44986127625842254, 0.28418549346016647, 0.8006341656757828)
0.8006341656757828
Loading the data ...
ELAGIGILTV
(0.6580569727891157, 0.5020727040816326, 0.9071003401360545)
0.9071003401360545
Loading the data ...
WLLWPVTLA
(0.08333333333333337, 0.375, 0.5416666666666666)
0.5416666666666666
Loading the data ...
NLVPMVATV
(0.4566615226337448, 0.4154449588477367, 0.7861689814814814)
0.7861689814814815
Loading the data ...
YLQPRTFLL
(0.6640946502057613, 0.618427069044353, 0.8885459533607681)
0.8885459533607681
Loading the data ...
SLFNTVATLY
(0.6292517006802721, 0.3299319727891156, 0.7040816326530612)
0.7040816326530612
Loading the data ...
RMFPNAPYL
(0.25, 0.45833333333333337, 0.75)
0.75
Loading the data ...
TLLFLMSFT
(0.20833333333333331, 0.416